In [2]:
pip install pandas


In [3]:
pip install keras


  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached absl_py-2.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached namex-0.1.0-py3-none-any.whl.metadata (322 bytes)
Using cached keras-3.10.0-py3-none-any.whl (1.4 MB)
Using cached absl_py-2.3.0-py3-none-any.whl (135 kB)
Using cached namex-0.1.0-py3-none-any.whl (5.9 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow


  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
    --------------------------------------- 5.0/376.0 MB 27.4 MB/s eta 0:00:14
   -- ------------------------------------- 22.8/376.0 MB 60.1 MB/s eta 0:00:06
   ---- ----------------------------------- 38.3/376.0 MB 65.8 MB/s eta 0:00:06
   ----- ---------------------------------- 50.3/37

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Leer CSV
df = pd.read_csv("simulacionEconomica.csv")


# Variables de entrada
X = df[[
    "demanda_preajuste",
    "inventario_preajuste",
    "produccion_preajuste",
    "precio_preajuste",
    "volatilidad_precio"
]]

# Objetivos
y_subida = df["subida_brusca_precio"]
y_escasez = df["escasez"]

# Escalar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_scaled, y_subida, test_size=0.2, random_state=42)
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(X_scaled, y_escasez, test_size=0.2, random_state=42)

# Crear modelo
def crear_modelo(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Modelos
modelo_subida = crear_modelo(X_train_s.shape[1])
modelo_escasez = crear_modelo(X_train_e.shape[1])

# Entrenar
modelo_subida.fit(X_train_s, y_train_s, epochs=20, batch_size=64, validation_split=0.2)
modelo_escasez.fit(X_train_e, y_train_e, epochs=20, batch_size=64, validation_split=0.2)

# Evaluar
print("Evaluación subida brusca de precio:", modelo_subida.evaluate(X_test_s, y_test_s))
print("Evaluación escasez:", modelo_escasez.evaluate(X_test_e, y_test_e))


Epoch 1/20


c:\Users\pinep\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


350/350 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7749 - loss: 0.4677 - val_accuracy: 0.8039 - val_loss: 0.4009
Epoch 2/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8019 - loss: 0.4062 - val_accuracy: 0.8086 - val_loss: 0.3894
Epoch 3/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8065 - loss: 0.4007 - val_accuracy: 0.8146 - val_loss: 0.3799
Epoch 4/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8066 - loss: 0.3912 - val_accuracy: 0.8170 - val_loss: 0.3777
Epoch 5/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8160 - loss: 0.3809 - val_accuracy: 0.8186 - val_loss: 0.3778
Epoch 6/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8133 - loss: 0.3870 - val_accuracy: 0.8112 - val_loss: 0.3749
Epoch 7/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8133 - loss: 0.3831 - val_accuracy: 0.8179 - val_loss: 0.3756
Epoch 8/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8170 - loss: 0.3791 - val_accuracy: 0.8186 - val_